# Sarter Notebook

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import haversine as hs

## Reading Data

In [106]:
# read train data
df_orders = pd.read_csv("orders.csv", sep=";")
df_products = pd.read_csv("product_attributes.csv", sep=",")
df_dists = pd.read_csv("cities_data.csv", sep=";")
df_ordersXproducts = pd.read_csv('ordersXproducts.csv')
del df_ordersXproducts['Unnamed: 0']

# read test data
df_test = pd.read_csv('finaltest.csv')
df_test['weight'] = df_test['weight'].fillna(df_test['weight'].mean())
df_test['late_order'] = True

## Merge tables

In [81]:
df_coord = df_dists[['city_from_name', 'city_from_coord']].drop_duplicates().reset_index().drop(columns=['index']).copy()

In [82]:
df_alt = df_dists.copy()

c2 = df_alt['city_from_name'].values.copy()
c1 = df_alt['city_to_name'].values.copy()
c4 = df_alt['city_from_coord'].values.copy()
c3 = df_alt['city_to_coord'].values.copy()

df_alt['city_from_name'] = c1
df_alt['city_to_name'] = c2
df_alt['city_from_coord'] = c3
df_alt['city_to_coord'] = c4

df_dists = pd.concat([df_dists, df_alt]).reset_index().drop(columns=['index'])

In [83]:
df = pd.merge(df_ordersXproducts, df_dists, how='left', left_on=['origin_port', 'logistic_hub'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_origin_hub'})

df = pd.merge(df, df_dists, how='left', left_on=['logistic_hub', 'customer'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_hub_customer'})

df = pd.merge(df, df_dists, how='left', left_on=['origin_port', 'customer'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_origin_customer'})

df = pd.merge(df, df_coord, how='left', left_on=['origin_port'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'origin_port_coord'})

df = pd.merge(df, df_coord, how='left', left_on=['logistic_hub'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'logistic_hub_coord'})

df = pd.merge(df, df_coord, how='left', left_on=['customer'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'customer_coord'})

In [84]:
for i in range(len(df)):
    if (not pd.isna(df.at[i, 'dist_origin_hub'])) and (not pd.isna(df.at[i, 'dist_hub_customer'])):
        df.at[i, 'dist_origin_customer'] = df.at[i, 'dist_origin_hub'] + df.at[i, 'dist_hub_customer']

In [87]:
df.isnull().sum()

order_id                   0
origin_port                0
3pl                        0
customs_procedures         0
logistic_hub            1049
customer                   0
product_id                 0
units                      0
late_order                 0
weight                   118
material_handling        118
dist_origin_hub         1049
dist_hub_customer       1049
dist_origin_customer       0
origin_port_coord          0
logistic_hub_coord      1049
customer_coord          1112
dtype: int64

In [85]:
x = df[~df['logistic_hub'].isna()]
indexes = x[x['dist_hub_customer'].isna()].index

for i in indexes:
    df.at[i, 'dist_hub_customer'] = 0

In [86]:
x = df[df['dist_origin_customer'].isna()]
indexes = x.index

for i in indexes:
    df.at[i, 'dist_origin_customer'] = 0

In [88]:
df

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,late_order,weight,material_handling,dist_origin_hub,dist_hub_customer,dist_origin_customer,origin_port_coord,logistic_hub_coord,customer_coord
0,366c7a3d298f,Rotterdam,v_002,DTP,Venlo,Marseille,1692723,583,1,1778.0,5.0,130.0459,902.0420,1032.0879,"(51.9244424, 4.47775)","(51.39244885, 6.1511724144122955)","(43.2961743, 5.3699525)"
1,45f906331e10,Rotterdam,v_004,CRF,Rome,Marseille,1644308,459,0,1088.0,3.0,1269.2365,604.0216,1873.2581,"(51.9244424, 4.47775)","(41.8933203, 12.4829321)","(43.2961743, 5.3699525)"
2,ac84a6e4af0f,Athens,v_002,CRF,Venlo,Paris,1684170,464,1,505.0,4.0,2029.5726,392.9249,2422.4975,"(37.9839412, 23.7283052)","(51.39244885, 6.1511724144122955)","(48.8588897, 2.3200410217200766)"
3,f5e98cb29790,Rotterdam,v_004,CRF,Lille,Milan,1620510,678,0,1308.0,4.0,173.9644,733.8784,907.8428,"(51.9244424, 4.47775)","(50.6365654, 3.0635282)","(45.4641943, 9.1896346)"
4,a9e7c9bee35b,Barcelona,v_002,CRF,Venlo,Berlin,1699372,353,0,1465.0,0.0,1153.4178,512.8188,1666.2366,"(41.3828939, 2.1774322)","(51.39244885, 6.1511724144122955)","(52.5170365, 13.3888599)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114271,3f4b15fb770e,Rotterdam,v_002,CRF,Dusseldorf,Bordeaux,1681376,645,0,1896.0,3.0,177.3075,896.2065,1073.5140,"(51.9244424, 4.47775)","(51.2254018, 6.7763137)","(44.841225, -0.5800364)"
114272,d2e6978a38fd,Barcelona,v_004,DTD,Dusseldorf,Berlin,1676942,502,0,746.0,1.0,1149.4065,477.3717,1626.7782,"(41.3828939, 2.1774322)","(51.2254018, 6.7763137)","(52.5170365, 13.3888599)"
114273,b88babd5c7bd,Rotterdam,v_002,DTP,Dusseldorf,Rome,1692737,464,0,572.0,5.0,177.3075,1125.0290,1302.3365,"(51.9244424, 4.47775)","(51.2254018, 6.7763137)","(41.8933203, 12.4829321)"
114274,b0b5c761613f,Barcelona,v_003,DTD,Dusseldorf,Munich,1699974,388,0,1894.0,1.0,1149.4065,487.5926,1636.9991,"(41.3828939, 2.1774322)","(51.2254018, 6.7763137)","(48.1371079, 11.5753822)"


In [89]:
df.to_csv('finaldf.csv', index=False)

In [ ]:
x = df[~df['logistic_hub'].isna()]
indexes = x[x['dist_hub_customer'].isna()].index

for i in indexes:
    df.at[i, 'dist_hub_customer'] = 0

In [ ]:
x = df[df['dist_origin_customer'].isna()]
indexes = x.index

for i in indexes:
    df.at[i, 'dist_origin_customer'] = 0

# Merge test

In [160]:
df_test = df_test.replace('ATHENAS', 'Athens')
df_test = df_test.replace('BCN', 'Barcelona')

In [161]:
df = pd.merge(df_test, df_products, how='left', left_on=['product_id'], right_on=['product_id'])

df = pd.merge(df, df_dists, how='left', left_on=['origin_port', 'logistic_hub'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_origin_hub'})

df = pd.merge(df, df_dists, how='left', left_on=['logistic_hub', 'customer'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_hub_customer'})

df = pd.merge(df, df_dists, how='left', left_on=['origin_port', 'customer'], right_on=['city_from_name', 'city_to_name'])
del df['city_from_name']
del df['city_to_name']
del df['city_from_coord']
del df['city_to_coord']
df = df.rename(columns={'distance': 'dist_origin_customer'})

df = pd.merge(df, df_coord, how='left', left_on=['origin_port'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'origin_port_coord'})

df = pd.merge(df, df_coord, how='left', left_on=['logistic_hub'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'logistic_hub_coord'})

df = pd.merge(df, df_coord, how='left', left_on=['customer'], right_on=['city_from_name'])
del df['city_from_name']
df = df.rename(columns={'city_from_coord': 'customer_coord'})

In [162]:
for i in range(len(df)):
    if (not pd.isna(df.at[i, 'dist_origin_hub'])) and (not pd.isna(df.at[i, 'dist_hub_customer'])):
        df.at[i, 'dist_origin_customer'] = df.at[i, 'dist_origin_hub'] + df.at[i, 'dist_hub_customer']

In [168]:
df.isnull().sum()

order_id                  0
origin_port               0
3pl                       0
customs_procedures        0
logistic_hub            265
customer                  0
product_id                0
units                     0
weight                   45
material_handling        45
dist_origin_hub         265
dist_hub_customer       265
dist_origin_customer      0
origin_port_coord         0
logistic_hub_coord      265
customer_coord          288
dtype: int64

In [165]:
x = df[~df['logistic_hub'].isna()]
indexes = x[x['dist_hub_customer'].isna()].index

for i in indexes:
    df.at[i, 'dist_hub_customer'] = 0

In [167]:
x = df[df['dist_origin_customer'].isna()]
indexes = x.index

for i in indexes:
    df.at[i, 'dist_origin_customer'] = 0

In [169]:
df.to_csv('finaltest.csv', index=False)

# Data preprocessing

In [204]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders.binary import BinaryEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

In [205]:
df = pd.read_csv('finaldf.csv')
not_useful = ['order_id', 'origin_port_coord',
       'logistic_hub_coord', 'customer_coord', 'logistic_hub', 'material_handling']
for col in not_useful:
    del df[col]
df['weight'] = df['weight'].fillna(df['weight'].mean())
df = df.dropna().reset_index().drop(columns=['index'])

In [206]:
units, weight = df['units'].values, df['weight'].values
df['total_weight']= units*weight

In [207]:
categorical_columns = ['origin_port', '3pl', 'customs_procedures', 'product_id', 'customer']
numerical_columns = ['total_weight', 'units', 'weight', 'dist_origin_customer']

target = df['late_order']
data = df.drop(columns=['late_order'])

In [208]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])



In [209]:
model = Pipeline(steps=[('preprocessor', preprocessor), 
                        ('classifier', LogisticRegression(max_iter=50))])
cv_results = cross_validate(model, data, target, cv=5)
cv_results['test_score'].mean()

C:\Users\thoma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\thoma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

0.8226748043319093

In [210]:
model['classifier'].coef_[0][-2:]

AttributeError: 'LogisticRegression' object has no attribute 'coef_'

In [189]:
num = 0
for col in categorical_columns:
    num += len(df[col].unique())
num

808

In [191]:
len(np.squeeze(np.asarray(preprocessor.fit_transform(data).todense()))[0])

811

In [173]:
np.squeeze(np.asarray(preprocessor.fit_transform(data).todense()))[0][-3:]

array([ 0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [163]:
x = np.squeeze(np.asarray(preprocessor.fit_transform(data).todense()))
y = list(target.values)
y = [[e] for e in y]
y = torch.tensor(y, dtype=torch.float32)

In [164]:
padding = np.zeros(1024-len(x[0]))

In [165]:
z = []
for i in range(len(x)):
    z.append(list(np.concatenate((x[i], padding))))
x = torch.tensor(z, dtype=torch.float32)

In [211]:
model.fit(data, target)

C:\Users\thoma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['origin_port', '3pl',
                                                   'customs_procedures',
                                                   'product_id', 'customer']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['total_weight', 'units',
                                                   'weight',
                                                   'dist_origin_customer'])])),
                ('classifier', LogisticRegression(max_iter=50))])

In [212]:
model['classifier'].coef_

array([[ 2.37607329e-01, -1.43954377e-01, -6.07072455e-01,
         5.47376384e-01, -6.45289311e-01, -2.14684190e-01,
        -2.00822386e-01, -5.64129769e-01, -1.59045498e-01,
         2.09755764e-01,  6.46201711e-02,  1.49131159e+00,
        -7.18783915e-01, -5.42722081e-02, -3.89862955e-01,
        -5.34291147e-01, -6.61715437e-02, -1.43272600e-01,
        -3.96985974e-01, -9.85899120e-01, -9.07160345e-02,
        -5.41839152e-01, -5.43056524e-01, -3.65868344e-02,
        -2.99508363e-01, -1.53679624e-01,  5.09532298e-01,
        -2.40987586e-01,  2.18171615e-01, -2.82120073e-01,
        -2.11179885e-01, -7.57320118e-01, -3.50086282e-01,
         2.02277737e+00, -7.49554693e-01, -2.37243611e-01,
        -3.53444289e-01, -5.14359697e-01, -1.60350814e-01,
        -3.53134356e-01, -2.74749578e-01, -9.41475641e-01,
        -2.19171631e-01,  2.21995788e-01,  1.28205766e+00,
        -7.64722497e-01, -2.23247072e-01, -6.96932194e-01,
         1.63999718e+00,  1.96240726e+00, -2.13006796e-0

In [154]:
LogisticRegressionProbs = model.predict_proba(df_test.drop(columns=not_useful+['late_order']))[:, 1]

In [155]:
LogisticRegressionProbs

array([0.1243    , 0.14126935, 0.23928753, ..., 0.12044055, 0.38198951,
       0.21665572])

In [156]:
df_pred_LR = pd.DataFrame()
df_pred_LR['order_id'] = df_test['order_id']
df_pred_LR['late_order'] = LogisticRegressionProbs

In [157]:
df_pred_LR.to_csv('pred_LogisticRegression4.csv', index=False)

# Papi DL

In [115]:
import math

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [406]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [332]:
d_model

1024

In [116]:
d_model = len(x[0])

class Net(nn.Module):
    """
    Text classifier based on a pytorch TransformerEncoder.
    """

    def __init__(
        self,
        nhead=4,
        dim_feedforward=1024,
        num_layers=4,
        dropout=0.1,
        activation="relu",
        classifier_dropout=0.1,
    ):

        super().__init__()

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
        )
        self.classifier = nn.Linear(d_model, 1)
        self.d_model = d_model

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = torch.sigmoid(self.classifier(x))

        return x

In [117]:
batch = 0
epoch_loss = 0
epoch_correct = 0
epoch_count = 0
predictions = model(x_train[batch*batch_size : (batch+1)*batch_size].to(device))
labels = y_train[batch*batch_size : (batch+1)*batch_size].to(device)

loss = criterion(predictions, labels)

correct = predictions.round() == labels
acc = correct.sum().item() / correct.size(0)

epoch_correct += correct.sum().item()
epoch_count += correct.size(0)

NameError: name 'x_train' is not defined

In [357]:
epoch_correct / epoch_count

0.9052734375

In [118]:
epochs = 4
model = Net().to(device)

criterion = nn.BCELoss()

lr = 1e-4
optimizer = torch.optim.Adam(
    (p for p in model.parameters() if p.requires_grad), lr=lr
)

torch.manual_seed(0)

batch_size=1024
batches = math.ceil(len(x)/batch_size)

print("starting")
for epoch in range(epochs):
    print(f"{epoch=}")
    epoch_loss = 0
    epoch_correct = 0
    epoch_count = 0
    for batch in range(math.ceil(len(x)/batch_size)):
        optimizer.zero_grad()
        predictions = model(x[batch*batch_size : (batch+1)*batch_size].to(device))
        labels = y[batch*batch_size : (batch+1)*batch_size].to(device)

        loss = criterion(predictions, labels)

        correct = predictions.round() == labels
        acc = correct.sum().item() / correct.size(0)

        epoch_correct += correct.sum().item()
        epoch_count += correct.size(0)

        epoch_loss += loss.item()

        loss.backward()

        optimizer.step()
    """
        predictions = model(x_train[batch*batch_size : (batch+1)*batch_size].to(device))
        labels = y_train[batch*batch_size : (batch+1)*batch_size].to(device)

        loss = criterion(predictions, labels)

        correct = predictions.round() == labels
        acc = correct.sum().item() / correct.size(0)

        epoch_correct += correct.sum().item()
        epoch_count += correct.size(0)

        epoch_loss += loss.item()

        loss.backward()

        optimizer.step()
    
    with torch.no_grad():
        test_epoch_loss = 0
        test_epoch_correct = 0
        test_epoch_count = 0

        for batch in range(math.ceil(len(x_test)/batch_size)):
            predictions = model(x_test[batch*batch_size : (batch+1)*batch_size].to(device))
            labels = y_test[batch*batch_size : (batch+1)*batch_size].to(device)
            test_loss = criterion(predictions, labels)

            correct = predictions.round() == labels
            acc = correct.sum().item() / correct.size(0)

            test_epoch_correct += correct.sum().item()
            test_epoch_count += correct.size(0)
            test_epoch_loss += loss.item()

    print(f"{epoch_loss=}")
    print(f"epoch accuracy: {epoch_correct / epoch_count}")
    print(f"{test_epoch_loss=}")
    print(f"test epoch accuracy: {test_epoch_correct / test_epoch_count}")
    """
    print(f"{epoch_loss=}")
    print(f"epoch accuracy: {epoch_correct / epoch_count}")

starting
epoch=0
epoch_loss=51.87281847000122
epoch accuracy: 0.7938537163267291
epoch=1
epoch_loss=45.233750373125076
epoch accuracy: 0.8235153701296979
epoch=2
epoch_loss=44.430546551942825
epoch accuracy: 0.8265036380836185
epoch=3
epoch_loss=43.90933087468147
epoch accuracy: 0.828775782652132


In [119]:
torch.save(model.state_dict(), 'PapiDL.model')

In [89]:
data = df_test.drop(columns=not_useful+['late_order'])

k = np.squeeze(np.asarray(preprocessor.fit_transform(data).todense()))

padding = np.zeros(1024-len(k[0]))

z = []
for i in range(len(k)):
    z.append(list(np.concatenate((k[i], padding))))
k = torch.tensor(z, dtype=torch.float32)

In [120]:
preds = []
batch_size=128
for batch in range(math.ceil(len(k)/batch_size)):
    preds.append(model(k[batch*batch_size : (batch+1)*batch_size].to(device)))

In [121]:
preds = torch.cat(list(torch.cat(preds))).detach().numpy()

In [124]:
def trans(num):
    return 1 - num
t = np.vectorize(trans)
p = t(p)

In [125]:
p

array([0.25205749, 0.30069175, 0.09594342, ..., 0.16871293, 0.19298422,
       0.08183663])

In [126]:
df_pred_DL = pd.DataFrame()
df_pred_DL['order_id'] = df_test['order_id']
df_pred_DL['late_order'] = p

In [127]:
df_pred_DL.to_csv('pred_DL.csv', index=False)